<h1> Multivariate Normal Mixture Model

In [1]:
ϕ(n,v,μ,Σ) = (2*π)^(-0.5*n) * det(Σ)^(-0.5) * exp(-0.5*(v-μ)'*inv(Σ)*(v-μ))

function EM(v, m,μ,Σ)
    #EM algorithm for multivariate normal mixture model with g mixture components
    #as described in Finite Mixture Models, p.82 (Geoffrey McLachlan & David Peel, 2000)

    n = size(v,2)
    g = length(Σ)
    τ = zeros(n,g)
    for i=1:g, j=1:n; τ[j,i] = m[i]*ϕ(n,v[:,j],μ[:,i],Σ[i]); end
    τ = τ./sum(τ,2)
    
    T1 = sum(τ,1)
    T2 = v*τ
    T3 = [sum([τ[j,i]*v[:,j]*v[:,j]' for j=1:n]) for i=1:g]
    
    m = T1./n
    μ = T2./T1
    Σ = [(T3[i] - 1/T1[i] * T2[:,i]*T2[:,i]')/T1[i] for i=1:g]
    return m,μ,Σ
end

function some_data()
    #some 2 dimensional data in 4 clusters
    v1=[1 0.5; 0.5 1]*randn(2,50)*0.4.+[-1,-1]
    v2=[1 -0.5; -0.5 1]*randn(2,50)*0.4.+[-1,+1]
    v3=[1 0.25; 0.25 1]*randn(2,50)*0.4.+[+1,-1]
    v4=[1 -0.25; -0.25 1]*randn(2,50)*0.4.+[+0.5,+0.5]
    v=hcat(v1,v2,v3,v4);
    return v
end

function plot_2d_data_and_components(v,m,μ,Σ)
    a=collect(0:0.1:2.1*π)
    circle=vcat(cos.(a)',sin.(a)')
    covplot=plot()
    pΣ = deepcopy(Σ) #plot with σ size instead of σ² size
    for i=1:length(Σ) #component
        pΣ[i][1,1]=sqrt(pΣ[i][1,1])
        pΣ[i][2,2]=sqrt(pΣ[i][2,2])
        cov=pΣ[i]*circle .+ μ[:,i]
        plot!(covplot,cov[1,:],cov[2,:], lab=string("c",i), width=2)
    end
    plot(covplot)
    scatter!(v[1,:],v[2,:], title=string("proportions: ",round.(m,2)), lab="data", color=:black)
end

plot_2d_data_and_components (generic function with 1 method)

<h2> Visualize learning the mixture component parameters

In [2]:
using Plots; gr(xlim=(-2,2), ylim=(-2,2), size=(600,600))

p=2 #dimensionality of data
t = some_data()

g=4 #nr of components
m=ones(g)/g
μ=randn(p,g)
Σ=[eye(p) for i=1:g]

@gif for ITER=1:100
    m,μ,Σ = EM(t, m,μ,Σ)
    plot_2d_data_and_components(t,m,μ,Σ)
end

INFO: Saved animation to /home/andy/dev/githubnotebooks/MixtureModeling/Multivariate_Gaussians/tmp.gif


Plots.AnimatedGif("/home/andy/dev/githubnotebooks/MixtureModeling/Multivariate_Gaussians/tmp.gif")

// Anders